In [1]:

from contextlib import nullcontext
import numpy as np
from numpy.core.numeric import outer
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy.core.fromnumeric import shape, size
import seaborn as sns


In [3]:
def Loss(Y, Y_predicted):
    return(sum(np.square(Y - Y_predicted))/Y.size)

def LossAbs(Y, Y_predicted):
    return(sum(np.absolute(Y - Y_predicted))/Y.size)


In [6]:
class Data:
    flag=0
    X_min=0
    X_max=0

    def outliers(self,X, Y):
        a = X
        mean, stdev = np.mean(a, axis=0), np.std(a, axis=0)
        outliers = ((np.abs(a[:,0] - mean[0]) > stdev[0])
                    * (np.abs(a[:,1] - mean[1]) > stdev[1])
                    * (np.abs(a[:,2] - mean[2]) > stdev[2]))
        # print(shape(a[~outliers]))
        # print (shape(Y[~outliers]))
        return X[~outliers], Y[~outliers]
    
    def normalization(self, X):
        self.X_min=X.min()
        self.X_max=X.max()
        return((X-X.min())/(X.max()-X.min()))

    #This function expect all indeces before lableIndex are feature indeces 
    def extract(self, Location, desierdNumFeauter, LableIndex):
        start=0
        myfile= pd.read_csv(Location)
        if(myfile.index[0]==id):
            start =1
        #Select feature function based on Pearson’s Correlation 
        X = myfile.iloc[:, start:desierdNumFeauter].values
        Y = myfile.iloc[:, LableIndex].values

        if(np.ptp(X, axis=0).max()-np.ptp(X, axis=0).min()>1000):
            X= self.normalization(X)
            self.flag=1
        return X, Y

    def normalize_Vector(self, X):
        return (X-self.X_min)/(self.X_max-self.X_min)


In [7]:
class Gradiant_Descent:  
    weight=0
    w_0=0
    normData=nullcontext
    def __init__(self, norm=nullcontext):
        self.normData=norm
        
    def fit(self, X, Y, learning_rate=0.0001, iterations=10000):
        self.weight = np.zeros(shape(X)[1])
        Y_pred= np.dot(X, self.weight)+self.w_0
        while(iterations):
            weightDef= -2*np.dot(np.transpose((Y-Y_pred)), X)
            w_0Def = -2*sum(Y-Y_pred)
            self.weight = self.weight-(learning_rate*weightDef)
            self.w_0 = self.w_0 - learning_rate*w_0Def
            Y_pred= np.dot(X, self.weight)+self.w_0
            iterations-=1
    
    def predict(self, X_test):
        Y_predict = np.dot(X_test, self.weight)+ self.w_0
        return Y_predict
    
    def predict_value(self ,X_vector):
        if(self.normData!=nullcontext and self.normData.flag==1):
            X_vector= self.normData.normalize_Vector(X_vector)

        Y_value=np.dot(X_vector, self.weight)+ self.w_0
        return Y_value

In [8]:
#Random Number
X = np.random.rand(1000, 3)
Y = np.dot(X, (5, 3, 1.5))+6 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y) 
myModel = Gradiant_Descent()
myModel.fit(X_train, Y_train, iterations=1000)
Y_predict= myModel.predict(X_test)
print()
print("Random data")
print("Random data generated model Loss {} ".format(np.around(Loss(Y_test, Y_predict), decimals=9)))
print("Housing price weight vector {} and w_0 {}".format(myModel.weight, myModel.w_0))
print("Predicted value based on vector [0.5 , 0.9, 0.7]: {} ".format(myModel.predict_value(np.array([0.5 , 0.9, 0.7]))))


Random data
Random data generated model Loss 0.0 
Housing price weight vector [5.00001552 3.00003578 1.50003336] and w_0 5.999956065529417
Predicted value based on vector [0.5 , 0.9, 0.7]: 12.250019385129526 


In [9]:
#House Price
myData = Data()
X, Y=myData.extract(r"E:\studying\3\ML\Labs\Task2\USA_Housing.csv", desierdNumFeauter= 3, LableIndex= 5)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y) 

X_noOut, Y_nonOut = myData.outliers(X_train, Y_train)

houseModel = Gradiant_Descent(myData)
houseModel.fit(X_noOut, Y_nonOut)
Y_predict= houseModel.predict(X_test)
print()
print("Housing price")
print("Housing price model Loss {} ".format(Loss(Y_test, Y_predict)))
print("Housing price weight vector {} and w_0 {}".format(houseModel.weight, houseModel.w_0))
X_value = np.array([63345.24005, 7.188236095, 5.586728665])
Y_pr=houseModel.predict_value(X_value)
print("Predicted value based on vector {} : {} ".format(X_value, Y_pr))


Housing price
Housing price model Loss 73757181515.8762 
Housing price weight vector [2265903.96915038   10472.7251149     8144.99573262] and w_0 -211989.87460963725
Predicted value based on vector [6.33452401e+04 7.18823609e+00 5.58672866e+00] : 1120688.905544591 
